# Import libs

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

# Load_split_train_test

In [2]:
data_dir = 'dataset/final_clean/'
def load_split_train_test(datadir, valid_size = .2):
    train_transforms = transforms.Compose([transforms.Resize(224),
                                       transforms.ToTensor(),
                                       ])
    test_transforms = transforms.Compose([transforms.Resize(224),
                                      transforms.ToTensor(),
                                      ])
    train_data = datasets.ImageFolder(datadir,       
                    transform=train_transforms)
    test_data = datasets.ImageFolder(datadir,
                    transform=test_transforms)
    num_train = len(train_data)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))
    np.random.shuffle(indices)
    from torch.utils.data.sampler import SubsetRandomSampler
    train_idx, test_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(test_idx)
    trainloader = torch.utils.data.DataLoader(train_data,
                   sampler=train_sampler, batch_size=1)
    testloader = torch.utils.data.DataLoader(test_data,
                   sampler=test_sampler, batch_size=1)
    return trainloader, testloader
trainloader, testloader = load_split_train_test(data_dir, .2)
print(trainloader.dataset.classes)

['AreaGraph', 'BarGraph', 'LineGraph', 'Map', 'ParetoChart', 'PieChart', 'RadarPlot', 'ScatterGraph', 'Table', 'VennDiagram']


# Device and network: resnet50   

### we can use other type of network like vgg16 or custom CNN based network

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() 
                                  else "cpu")
model = models.resnet50(pretrained=True)
device

device(type='cuda')

# FC and optimizer

In [5]:
for param in model.parameters():
    param.requires_grad = False
    
model.fc = nn.Sequential(nn.Linear(2048, 512),
                                 nn.ReLU(),
                                 nn.Dropout(0.2),
                                 nn.Linear(512, 10),
                                 nn.LogSoftmax(dim=1))
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.003)
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

# Training 

In [6]:
epochs = 1
steps = 0
running_loss = 0
print_every = 10
train_losses, test_losses = [], []
for epoch in range(epochs):
    
    for inputs, labels in trainloader:
        try:
            steps += 1
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            logps = model.forward(inputs)
            loss = criterion(logps, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

            if steps % print_every == 0:
                test_loss = 0
                accuracy = 0
                model.eval()
                with torch.no_grad():
                    for inputs, labels in testloader:
                        inputs, labels = inputs.to(device),labels.to(device)
                        logps = model.forward(inputs)
                        batch_loss = criterion(logps, labels)
                        test_loss += batch_loss.item()

                        ps = torch.exp(logps)
                        top_p, top_class = ps.topk(1, dim=1)
                        equals = top_class == labels.view(*top_class.shape)
                        accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
                train_losses.append(running_loss/len(trainloader))
                test_losses.append(test_loss/len(testloader))                    
                print(f"Epoch {epoch+1}/{epochs}.. "
                      f"Train loss: {running_loss/print_every:.3f}.. "
                      f"Test loss: {test_loss/len(testloader):.3f}.. "
                      f"Test accuracy: {accuracy/len(testloader):.3f}")
                running_loss = 0
                model.train()
            
        except:
            pass
torch.save(model, 'savedmodel.pth')

Epoch 1/1.. Train loss: 6.813.. Test loss: 3.428.. Test accuracy: 0.094
Epoch 1/1.. Train loss: 3.592.. Test loss: 3.713.. Test accuracy: 0.135
Epoch 1/1.. Train loss: 4.069.. Test loss: 2.661.. Test accuracy: 0.140
Epoch 1/1.. Train loss: 2.632.. Test loss: 2.429.. Test accuracy: 0.117
Epoch 1/1.. Train loss: 2.339.. Test loss: 2.480.. Test accuracy: 0.117
Epoch 1/1.. Train loss: 2.547.. Test loss: 2.350.. Test accuracy: 0.099
Epoch 1/1.. Train loss: 2.461.. Test loss: 2.302.. Test accuracy: 0.094
Epoch 1/1.. Train loss: 2.297.. Test loss: 2.303.. Test accuracy: 0.070
Epoch 1/1.. Train loss: 2.290.. Test loss: 2.303.. Test accuracy: 0.099
Epoch 1/1.. Train loss: 2.315.. Test loss: 2.302.. Test accuracy: 0.082
Epoch 1/1.. Train loss: 2.293.. Test loss: 2.301.. Test accuracy: 0.088
Epoch 1/1.. Train loss: 2.347.. Test loss: 2.437.. Test accuracy: 0.135
Epoch 1/1.. Train loss: 2.595.. Test loss: 2.300.. Test accuracy: 0.088
Epoch 1/1.. Train loss: 2.275.. Test loss: 2.298.. Test accuracy